<a href="https://colab.research.google.com/github/mbrady4/DS-Unit-1-Sprint-2-Data-Wrangling/blob/master/module2-join-datasets/LS_DS_122_Join_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_Lambda School Data Science_

# Join datasets

Objectives
- concatenate data with pandas
- merge data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
  - Combine Data Sets: Standard Joins
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join

## Download data

We’ll work with a dataset of [3 Million Instacart Orders, Open Sourced](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2)!

In [1]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2019-03-26 19:13:41--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.9.45
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.9.45|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’

instacart_online_gr 100%[===================>] 196.03M  64.8MB/s    in 3.0s    

2019-03-26 19:13:50 (64.8 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’ saved [205548478/205548478]



In [2]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [3]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01


## Goal: Reproduce this example

The first two orders for user id 1:

In [4]:
from IPython.display import display, Image
url = 'https://cdn-images-1.medium.com/max/1600/1*vYGFQCafJtGBBX5mbl0xyw.png'
example = Image(url=url, width=600)

display(example)

## Load data

Here's a list of all six CSV filenames

In [5]:
!ls -lh

total 681M
-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


For each CSV
- Load it with pandas
- Look at the dataframe's shape
- Look at its head (first rows)
- `display(example)`
- Which columns does it have in common with the example we want to reproduce?

In [0]:
import pandas as pd

### aisles

In [7]:
aisles = pd.read_csv('aisles.csv')
aisles.shape

(134, 2)

In [8]:
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


**No columns from aisles will be needed**

### departments

In [9]:
departments = pd.read_csv('departments.csv')
departments.shape

(21, 2)

In [10]:
departments.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


**No columns from departments dataframe will be needed**

### order_products__prior

In [11]:
order_products_prior = pd.read_csv('order_products__prior.csv')
order_products_prior.shape

(32434489, 4)

In [12]:
order_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


**Columns needed from Order_Products_Prior**
- order id
- product id
- add to cart order


### order_products__train

In [14]:
order_products_train = pd.read_csv('order_products__train.csv')
order_products_train.shape

(1384617, 4)

In [15]:
order_products_train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


**Columns needed from Order_Products_Train:**
- order id
- product id 
- add to cart order

### orders

In [16]:
orders = pd.read_csv('orders.csv')
orders.shape

(3421083, 7)

In [17]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


**Columns from Orders dataframe**
- order id
- user id
- order number 
- order dow
- order hour of day

### products

In [18]:
products = pd.read_csv('products.csv')
products.shape

(49688, 4)

In [19]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


**Columns from Products dataframe:**
- product id
- product name

## Concatenate order_products__prior and order_products__train

In [0]:
order_products = pd.concat([order_products_prior, order_products_train])
row_a, col_a = order_products_train.shape
row_b, col_b = order_products_prior.shape
row_c, col_c = order_products.shape

assert row_a + row_b == row_c

In [23]:
order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


## Get a subset of orders — the first two orders for user id 1

In [28]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [56]:
condition = (orders['user_id'] == 1) & (orders['order_number'] <= 2)

columns = ['user_id', 
           'order_id', 
           'order_number', 
           'order_dow', 
           'order_hour_of_day']

subset = orders.loc[condition, columns]
subset

,user_id,order_id,order_number,order_dow,order_hour_of_day
0,1,2539329,1,2,8
1,1,2398795,2,3,7


## Merge dataframes

**Need to merge subset with order_products dataframe (need add to cart order and product id from order_products)**

In [57]:
merged = pd.merge(subset, order_products, how='inner', on='order_id')
merged.head()

,user_id,order_id,order_number,order_dow,order_hour_of_day,product_id,add_to_cart_order,reordered
0,1,2539329,1,2,8,196,1,0
1,1,2539329,1,2,8,14084,2,0
2,1,2539329,1,2,8,12427,3,0
3,1,2539329,1,2,8,26088,4,0
4,1,2539329,1,2,8,26405,5,0


In [0]:
merged = merged.drop(columns=['reordered'])

In [59]:
merged = pd.merge(merged, products[['product_name', 'product_id']], how='inner', on='product_id')
merged.head(11)

,user_id,order_id,order_number,order_dow,order_hour_of_day,product_id,add_to_cart_order,product_name
0,1,2539329,1,2,8,196,1,Soda
1,1,2398795,2,3,7,196,1,Soda
2,1,2539329,1,2,8,14084,2,Organic Unsweetened Vanilla Almond Milk
3,1,2539329,1,2,8,12427,3,Original Beef Jerky
4,1,2398795,2,3,7,12427,3,Original Beef Jerky
5,1,2539329,1,2,8,26088,4,Aged White Cheddar Popcorn
6,1,2398795,2,3,7,26088,5,Aged White Cheddar Popcorn
7,1,2539329,1,2,8,26405,5,XL Pick-A-Size Paper Towel Rolls
8,1,2398795,2,3,7,10258,2,Pistachios
9,1,2398795,2,3,7,13176,4,Bag of Organic Bananas


In [0]:
merged = merged.sort_values(by=['order_number', 'add_to_cart_order'])

In [0]:
merged.columns = [ column.replace('_',' ') for column in merged ]

In [62]:
merged

,user id,order id,order number,order dow,order hour of day,product id,add to cart order,product name
0,1,2539329,1,2,8,196,1,Soda
2,1,2539329,1,2,8,14084,2,Organic Unsweetened Vanilla Almond Milk
3,1,2539329,1,2,8,12427,3,Original Beef Jerky
5,1,2539329,1,2,8,26088,4,Aged White Cheddar Popcorn
7,1,2539329,1,2,8,26405,5,XL Pick-A-Size Paper Towel Rolls
1,1,2398795,2,3,7,196,1,Soda
8,1,2398795,2,3,7,10258,2,Pistachios
4,1,2398795,2,3,7,12427,3,Original Beef Jerky
9,1,2398795,2,3,7,13176,4,Bag of Organic Bananas
6,1,2398795,2,3,7,26088,5,Aged White Cheddar Popcorn


In [67]:
columsTitles = ['user id', 'order id', 'order number', 'order dow', 'order hour of day', 'add to cart order', 'product id', 'product name']

merged = merged.reindex(columns=columsTitles)

merged

,user id,order id,order number,order dow,order hour of day,add to cart order,product id,product name
0,1,2539329,1,2,8,1,196,Soda
2,1,2539329,1,2,8,2,14084,Organic Unsweetened Vanilla Almond Milk
3,1,2539329,1,2,8,3,12427,Original Beef Jerky
5,1,2539329,1,2,8,4,26088,Aged White Cheddar Popcorn
7,1,2539329,1,2,8,5,26405,XL Pick-A-Size Paper Towel Rolls
1,1,2398795,2,3,7,1,196,Soda
8,1,2398795,2,3,7,2,10258,Pistachios
4,1,2398795,2,3,7,3,12427,Original Beef Jerky
9,1,2398795,2,3,7,4,13176,Bag of Organic Bananas
6,1,2398795,2,3,7,5,26088,Aged White Cheddar Popcorn


# Assignment

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

## Stretch challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.